# MEE-MVR desalination

A multistage evaporator with mechanical vapor recompression has been built in Pyomo. This notebook demonstrates how to build the model, specify inlet conditions and obtain CAPEX and OPEX post optimization.
MEE-MVR model is taken from Onishi's 2017 paper on multi-effect evaporators[1].

<br>
We will begin with the relevant imports:
<br>
1) Pyomo for developing the model 
<br>
2) degrees_of_freedom package from IDAES core to analyse the degrees of freedom
<br>
3) mee_mvr model from PARETO

In [1]:
import pyomo.environ as pyo
from idaes.core.util.model_statistics import degrees_of_freedom
from pareto.models_extra.desalination_models.mee_mvr import make_mee_mvr_model

The inputs to the MEE-MVR process are:
<br>
1) N_evap - An integer indicating the number of evaporator stages (default = 1)
<br>
2) inputs_variables - A boolean indicating if the feed and salinity to the desalination unit are variables (default = False)
<br>
In this notebook, we will always assume our inputs are parameters (inputs_variables = False). 

In [4]:
N_evap = 1
m = make_mee_mvr_model(N_evap = N_evap)
print("Build succesful")

Build succesful


To specify the inlet feed flow and salinity, we set the flow_feed and salt_feed to the desired values. In this example we set feed flowrate to 10 kg/s and feed salinity to 100 g/kg.

In [14]:
m.flow_feed = 10
m.salt_feed = 100

The number of degrees of freedom can be checked using the deegrees_of_freedom function in IDAES

In [15]:
print("DOF = ", degrees_of_freedom(m))

DOF =  2


The MEE-MVR model has 2 degrees of freedom. We will first fix these two degrees of freedom to run a simulation. The evaporator area is fixed to 300 sq. meter and compressor capacity is fixed to 2000 hp

In [17]:
m.each_evaporator_area[0].fix(300)
m.compressor_capacity.fix(2000)

# Solving the simulation
We will use the open source non-linear solver IPOPT available through Pyomo. The following block instantiates IPOPT and solves the MEE-MVR simulation

In [18]:
ipopt = pyo.SolverFactory('ipopt')
ipopt.solve(m, tee= True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 55, 'Number of variables': 49, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.06006026268005371}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

The objective value can be obtained from the solved model. 

In [19]:
print("Objective function =", pyo.value(m.obj))
print("preheater area = ", pyo.value(m.preheater_area))

Objective function 2461.5094185697653
preheater area 1.5273756300662116


Now we unfix the degrees of freedom to solve the optimization problem. We will minimize the total annualized cost and let the optimizer decide the optimal evaporator area, and compressor capacity

In [20]:
m.each_evaporator_area[0].unfix()
m.compressor_capacity.unfix()
ipopt.solve(m, tee= True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 55, 'Number of variables': 51, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.04599905014038086}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [22]:
print("Objective function = ", pyo.value(m.obj))
print("preheater area = ", pyo.value(m.preheater_area))
print("Evaporator area = ", pyo.value(m.each_evaporator_area[0]))
print("Compressor capacity = ", pyo.value(m.compressor_capacity))

Objective function 1452.031184108574
preheater area 51.70148043955233
Evaporator area 372.0
Compressor capacity 1109.1581231287237
